In [2]:
import pandas as pd
import numpy as np

In [175]:
file_path = 'data/Concentration Demographics Example.xlsx'
xls = pd.ExcelFile(file_path)
sheet_names = xls.sheet_names[:-1]
all_sheets = {}

for sheet_name in sheet_names:
    print(f"--------Processing sheet: {sheet_name}--------")
    all_sheets[sheet_name] = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=1, header=[2])
    print(all_sheets[sheet_name].head(1))

--------Processing sheet: Applied Math.-Computer Sci.--------
   ID  Name              Status  Confidentiality Ind?  \
0 NaN   NaN  Active / On Campus                   NaN   

  Combined or Concurrent Degree Degree  Sem Level Sex Race/Ethnicity  HUG?  \
0                           NaN    SCB          7   M           Asian  NaN   

  First Gen? Concentration 2 Concentration 3 Certificate Program Name  \
0        NaN             NaN             NaN                      NaN   

  Intended Completion Term  
0                Fall 2024  
--------Processing sheet: Computational Biology--------
   ID  Name              Status  Confidentiality Ind?  \
0 NaN   NaN  Active / On Campus                   NaN   

   Combined or Concurrent Degree Degree  Sem Level Sex Race/Ethnicity  HUG?  \
0                            NaN     AB          6   F           Asian  NaN   

  First Gen? Concentration 2  Concentration 3 Certificate Program Name  \
0        NaN             NaN              NaN            

In [76]:
# file_path = 'data/Concentration Demographics Example.xlsx'
# xls = pd.ExcelFile(file_path)
# sheet_names = xls.sheet_names[:-1]

# all_sheets = pd.read_excel(file_path, sheet_name=None, skiprows=1, header=[2])
# processed_dataframe = {}

# for sheet_name, data in all_sheets.items():
#     print(f"--------Processing sheet: {sheet_name}--------")
#     processed_dataframe[sheet_name] = data
#     print(data.head(1))

In [73]:
all_sheets.sheet_names

AttributeError: 'dict' object has no attribute 'sheet_names'

In [176]:
for sheet_name, df in all_sheets.items():
    # Add a new column to each DataFrame with the name of the sheet
    df['SheetName'] = sheet_name

In [177]:
# combined the dataframes
combined_df = pd.concat(all_sheets.values(), ignore_index=True)
combined_df = combined_df.set_index('SheetName')
print(combined_df.shape)
combined_df.head()

(893, 15)


,ID,Name,Status,Confidentiality Ind?,Combined or Concurrent Degree,Degree,Sem Level,Sex,Race/Ethnicity,HUG?,First Gen?,Concentration 2,Concentration 3,Certificate Program Name,Intended Completion Term
SheetName,,,,,,,,,,,,,,,
Applied Math.-Computer Sci.,NaN,NaN,Active / On Campus,NaN,NaN,SCB,7,M,Asian,NaN,NaN,NaN,NaN,NaN,Fall 2024
Applied Math.-Computer Sci.,NaN,NaN,Active / On Campus,NaN,NaN,SCB,8,M,Race/Ethnicity Unknown,NaN,NaN,Economics,NaN,NaN,Spring 2024
Applied Math.-Computer Sci.,NaN,NaN,Active / On Campus,NaN,NaN,SCB,8,M,Non-Resident Alien,NaN,NaN,NaN,NaN,NaN,Spring 2024
Applied Math.-Computer Sci.,NaN,NaN,Active / On Campus,NaN,SCBSCM,SCM,8,M,Race/Ethnicity Unknown,NaN,NaN,Computer Science,NaN,NaN,Spring 2024
Applied Math.-Computer Sci.,NaN,NaN,Active / On Campus,NaN,NaN,SCB,6,M,White,NaN,Y,NaN,NaN,NaN,Spring 2025


In [17]:
# concentration abbrev.
concentration_dict = {
    'APMA-CS': "Applied Math.-Computer Sci." ,
    'Comp Bio' : "Computational Biology" ,
    'CS' : "Computer Science" ,
    'CS-Econ' : "Computer Science-Economics" ,
    'Math-CS' : "Mathematics-Computer Science" ,
    'Cybersecurity' : "Cybersecurity" ,
}

In [79]:
# df = processed_dataframe['Applied Math.-Computer Sci.']
# # df.columns = df.columns.droplevel(0)
# print(df.columns)
# df.head()


In [19]:
print(df.dropna(axis=1, how='all').columns)

Index(['Status', 'Combined or Concurrent Degree', 'Degree', 'Sem Level', 'Sex',
       'Race/Ethnicity ', 'HUG?', 'First Gen?', 'Concentration 2',
       'Concentration 3', 'Certificate Program Name',
       'Intended Completion Term'],
      dtype='object')


In [178]:
def combine_col_names_values(row):
    return ', '.join([f"{col}={val}" for col, val in row.items()])

category_counts = {}
for col in ['Status', 'Combined or Concurrent Degree', 'Degree', 'Sem Level', 'Sex',
       'Race/Ethnicity ', 'HUG?', 'First Gen?', 'Concentration 2',
       'Concentration 3', 'Certificate Program Name',
       'Intended Completion Term']:
    pivot_df = combined_df.pivot_table(index=['SheetName', col],aggfunc='size', fill_value=0).reset_index(name='Count')
    pivot_df = pd.DataFrame(pivot_df).reset_index()
    # pivot_df.columns = pd.MultiIndex.from_product([[col], pivot_df.columns])
    pivot_df = pivot_df.pivot_table(index=['SheetName'], columns=[col], values='Count', fill_value=0)
    new_column_tuples = [(col, col_name) for col_name in pivot_df.columns]
    multi_index = pd.MultiIndex.from_tuples(new_column_tuples)

    # Assign this MultiIndex to the DataFrame's columns
    pivot_df.columns = multi_index
    pivot_df.reset_index(inplace=True)
    # new_levels = [pivot_df.index.levels[0], pivot_df.index.levels[1]]
    # print(new_levels)
    # pivot_df.index = pivot_df.columns.set_levels(col, level=[0, 1])

    # # Assign the new MultiIndex to the DataFrame    
    # df.index = new_index

    # pivot_df = pivot_df.set_index(['SheetName', col])
    # pivot_df = pivot_df.set_index(col).T
    category_counts[col] = pivot_df
# category_counts
pivot_df
# print(pivot_df.columns)

SheetName Intended Completion Term            \
                                               Fall 2024 Fall 2025   
0   Applied Math.-Computer Sci.                        3         1   
1         Computational Biology                        2         0   
2              Computer Science                       27         6   
3    Computer Science-Economics                        0         0   
4                 Cybersecurity                        0         0   
5  Mathematics-Computer Science                        0         0   

                                       
  Spring 2024 Spring 2025 Spring 2026  
0          63          81           6  
1          12          26           1  
2         281         235           9  
3          44          48           0  
4           1           0           0  
5          29          16           2

In [183]:
# Start with an empty DataFrame or initialize with the first DataFrame
combined_df = None

for name, df in category_counts.items():
    if combined_df is None:
        combined_df = df
    else:
        # Merge each DataFrame on the 'Key' column
        combined_df = pd.merge(combined_df, df, on='SheetName', how='left')

# Print the result
combined_df

/var/folders/2g/xtqrfmsd0nb3qhxzhnrtp3_h0000gn/T/ipykernel_60516/1224054544.py:9: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  combined_df = pd.merge(combined_df, df, on='SheetName', how='left')
/var/folders/2g/xtqrfmsd0nb3qhxzhnrtp3_h0000gn/T/ipykernel_60516/1224054544.py:9: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  combined_df = pd.merge(combined_df, df, on='SheetName', how='left')
/var/folders/2g/xtqrfmsd0nb3qhxzhnrtp3_h0000gn/T/ipykernel_60516/1224054544.py:9: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  combined_df = pd.merge(combined_df, df, on='SheetName', how='left')
/var/folders/2g/xtqrfmsd0nb3qhxzhnrtp3_h0000gn/T/ipykernel_60516/1224054544.py:9: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  combined_df = pd.mer

SheetName                                Status  \
                                Active / Enrolled Abroad Home Tuition   
0   Applied Math.-Computer Sci.                                     9   
1         Computational Biology                                     1   
2              Computer Science                                    14   
3    Computer Science-Economics                                     7   
4                 Cybersecurity                                     0   
5  Mathematics-Computer Science                                     2   

                                                                       \
  Active / Enrolled Domestic Study Away Active / Enrolled on Exchange   
0                                     1                             1   
1                                     0                             0   
2                                     0                            18   
3                                     0                             4   
4                                     0                             0   
5                                     0                             0   

                     Combined or Concurrent Degree                 Degree  \
  Active / On Campus                         ABSCB ABSCBSCM SCBSCM     AB   
0                143                           1.0      0.0    7.0      0   
1                 40                           NaN      NaN    NaN     17   
2                526                           0.0      4.0   27.0    210   
3                 81                           0.0      0.0    2.0     25   
4                  1                           0.0      0.0    1.0      0   
5                 45                           0.0      0.0    5.0      0   

         ... Concentration 3             Certificate Program Name  \
  ABSCB  ...         Physics Visual Arts      Engaged Scholarship   
0     1  ...             0.0         0.0                      0.0   
1     0  ...             NaN         NaN                      0.0   
2     0  ...             2.0         2.0                      1.0   
3     0  ...             0.0         0.0                      0.0   
4     0  ...             NaN         NaN                      NaN   
5     0  ...             NaN         NaN                      NaN   

                                            Intended Completion Term  \
  Entrepreneurship Intercultural Competence                Fall 2024   
0              1.0                      0.0                        3   
1              0.0                      1.0                        2   
2              4.0                      0.0                       27   
3              2.0                      0.0                        0   
4              NaN                      NaN                        0   
5              NaN                      NaN                        0   

                                                 
  Fall 2025 Spring 2024 Spring 2025 Spring 2026  
0         1          63          81           6  
1         0          12          26           1  
2         6         281         235           9  
3         0          44          48           0  
4         0           1           0           0  
5         0          29          16           2  

[6 rows x 106 columns]

In [194]:
result = combined_df.groupby(axis=1, level=0).sum()
combined_df['Total'] = result.iloc[:,-1]
combined_df

SheetName                                Status  \
                                Active / Enrolled Abroad Home Tuition   
0   Applied Math.-Computer Sci.                                     9   
1         Computational Biology                                     1   
2              Computer Science                                    14   
3    Computer Science-Economics                                     7   
4                 Cybersecurity                                     0   
5  Mathematics-Computer Science                                     2   

                                                                       \
  Active / Enrolled Domestic Study Away Active / Enrolled on Exchange   
0                                     1                             1   
1                                     0                             0   
2                                     0                            18   
3                                     0                             4   
4                                     0                             0   
5                                     0                             0   

                     Combined or Concurrent Degree                 Degree  \
  Active / On Campus                         ABSCB ABSCBSCM SCBSCM     AB   
0                143                           1.0      0.0    7.0      0   
1                 40                           NaN      NaN    NaN     17   
2                526                           0.0      4.0   27.0    210   
3                 81                           0.0      0.0    2.0     25   
4                  1                           0.0      0.0    1.0      0   
5                 45                           0.0      0.0    5.0      0   

         ... Concentration 3 Certificate Program Name                   \
  ABSCB  ...     Visual Arts      Engaged Scholarship Entrepreneurship   
0     1  ...             0.0                      0.0              1.0   
1     0  ...             NaN                      0.0              0.0   
2     0  ...             2.0                      1.0              4.0   
3     0  ...             0.0                      0.0              2.0   
4     0  ...             NaN                      NaN              NaN   
5     0  ...             NaN                      NaN              NaN   

                           Intended Completion Term                        \
  Intercultural Competence                Fall 2024 Fall 2025 Spring 2024   
0                      0.0                        3         1          63   
1                      1.0                        2         0          12   
2                      0.0                       27         6         281   
3                      0.0                        0         0          44   
4                      NaN                        0         0           1   
5                      NaN                        0         0          29   

                          Total  
  Spring 2025 Spring 2026        
0          81           6   154  
1          26           1    41  
2         235           9   558  
3          48           0    92  
4           0           0     1  
5          16           2    47  

[6 rows x 107 columns]

In [213]:
combined_df_percent = combined_df.copy()
combined_df_percent = combined_df_percent.fillna(0)
new_header = [f'{i} {j}' if i != j else f'{i}' for i, j in combined_df_percent.columns]
combined_df_percent.columns = new_header
combined_df_percent.set_index(['SheetName '], inplace=True)

In [214]:
# calculate the percentage
# convert to percentage
for column in combined_df_percent.columns:
    if column != 'Total ':
        combined_df_percent[column] = ((combined_df_percent[column] / combined_df_percent['Total ']) * 100).round(2)
combined_df_percent.head()

,Status Active / Enrolled Abroad Home Tuition,Status Active / Enrolled Domestic Study Away,Status Active / Enrolled on Exchange,Status Active / On Campus,Combined or Concurrent Degree ABSCB,Combined or Concurrent Degree ABSCBSCM,Combined or Concurrent Degree SCBSCM,Degree AB,Degree ABSCB,Degree SCB,...,Concentration 3 Visual Arts,Certificate Program Name Engaged Scholarship,Certificate Program Name Entrepreneurship,Certificate Program Name Intercultural Competence,Intended Completion Term Fall 2024,Intended Completion Term Fall 2025,Intended Completion Term Spring 2024,Intended Completion Term Spring 2025,Intended Completion Term Spring 2026,Total
SheetName,,,,,,,,,,,,,,,,,,,,,
Applied Math.-Computer Sci.,5.84,0.65,0.65,92.86,0.65,0.00,4.55,0.00,0.65,94.81,...,0.00,0.00,0.65,0.00,1.95,0.65,40.91,52.60,3.90,154
Computational Biology,2.44,0.00,0.00,97.56,0.00,0.00,0.00,41.46,0.00,58.54,...,0.00,0.00,0.00,2.44,4.88,0.00,29.27,63.41,2.44,41
Computer Science,2.51,0.00,3.23,94.27,0.00,0.72,4.84,37.63,0.00,56.81,...,0.36,0.18,0.72,0.00,4.84,1.08,50.36,42.11,1.61,558
Computer Science-Economics,7.61,0.00,4.35,88.04,0.00,0.00,2.17,27.17,0.00,70.65,...,0.00,0.00,2.17,0.00,0.00,0.00,47.83,52.17,0.00,92
Cybersecurity,0.00,0.00,0.00,100.00,0.00,0.00,100.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,100.00,0.00,0.00,1


In [215]:
combined_df_percent.to_csv('results/concetration.csv', index=True)

In [163]:
# Extract all DataFrames from the dictionary values
dataframes_list = list(category_counts.values())

# Concatenate all DataFrames vertically
concatenated_df = pd.concat(dataframes_list)

# Reset index if you want a clean index
concatenated_df = concatenated_df.reset_index(drop=True)

concatenated_df

SheetName                                Status  \
                                 Active / Enrolled Abroad Home Tuition   
0    Applied Math.-Computer Sci.                                   9.0   
1          Computational Biology                                   1.0   
2               Computer Science                                  14.0   
3     Computer Science-Economics                                   7.0   
4                  Cybersecurity                                   0.0   
..                           ...                                   ...   
59         Computational Biology                                   NaN   
60              Computer Science                                   NaN   
61    Computer Science-Economics                                   NaN   
62                 Cybersecurity                                   NaN   
63  Mathematics-Computer Science                                   NaN   

                                                                        \
   Active / Enrolled Domestic Study Away Active / Enrolled on Exchange   
0                                    1.0                           1.0   
1                                    0.0                           0.0   
2                                    0.0                          18.0   
3                                    0.0                           4.0   
4                                    0.0                           0.0   
..                                   ...                           ...   
59                                   NaN                           NaN   
60                                   NaN                           NaN   
61                                   NaN                           NaN   
62                                   NaN                           NaN   
63                                   NaN                           NaN   

                                                     ...  \
   Active / On Campus ABSCB ABSCBSCM SCBSCM  AB SCB  ...   
0               143.0   NaN      NaN    NaN NaN NaN  ...   
1                40.0   NaN      NaN    NaN NaN NaN  ...   
2               526.0   NaN      NaN    NaN NaN NaN  ...   
3                81.0   NaN      NaN    NaN NaN NaN  ...   
4                 1.0   NaN      NaN    NaN NaN NaN  ...   
..                ...   ...      ...    ...  ..  ..  ...   
59                NaN   NaN      NaN    NaN NaN NaN  ...   
60                NaN   NaN      NaN    NaN NaN NaN  ...   
61                NaN   NaN      NaN    NaN NaN NaN  ...   
62                NaN   NaN      NaN    NaN NaN NaN  ...   
63                NaN   NaN      NaN    NaN NaN NaN  ...   

                                                                      \
   Hispanic Liter. and Culture Medieval Cultures Engaged Scholarship   
0                          NaN               NaN                 NaN   
1                          NaN               NaN                 NaN   
2                          NaN               NaN                 NaN   
3                          NaN               NaN                 NaN   
4                          NaN               NaN                 NaN   
..                         ...               ...                 ...   
59                         NaN               NaN                 NaN   
60                         NaN               NaN                 NaN   
61                         NaN               NaN                 NaN   
62                         NaN               NaN                 NaN   
63                         NaN               NaN                 NaN   

                                                                              \
   Entrepreneurship Intercultural Competence Fall 2024 Fall 2025 Spring 2024   
0               NaN                      NaN       NaN       NaN         NaN   
1               NaN                      NaN       NaN       NaN         NaN   
2               NaN                      NaN       NaN     

In [25]:
# res_df = df.set_index('Applied Math.-Computer Sci.')
for col in ['Status', 'Combined or Concurrent Degree', 'Degree', 'Sem Level', 'Sex',
       'Race/Ethnicity ', 'HUG?', 'First Gen?', 'Concentration 2',
       'Concentration 3', 'Certificate Program Name',
       'Intended Completion Term']:
        if col in df.columns:
            pivot_df = df.pivot_table(index=['Applied Math.-Computer Sci.'], columns=[col], aggfunc='sum', fill_value=0)

            # Rename the pivoted columns to include the original column name
            pivot_df.columns = [f'{col} {category}' for category in pivot_df.columns]

            # Join the pivot results with the accumulator DataFrame
        #     pivot_accumulator = pivot_accumulator.join(pivot_df, on='Applied Math.-Computer Sci.')

pivot_df.head()

KeyError: 'Applied Math.-Computer Sci.'

In [ ]:
def pivot_column(data, pivot_columns, sheet_name):
    res_df = df[[sheet_name]].drop_duplicates().set_index('SheetName')

    for col in pivot_columns:
        if col in df.columns:
            pivot_df = df.pivot_table(index='SheetName', columns=col, values='Count', aggfunc='sum', fill_value=0)

            # Rename the pivoted columns to include the original column name
            pivot_df.columns = [f'{col} {category}' for category in pivot_df.columns]

            # Join the pivot results with the accumulator DataFrame
            pivot_accumulator = pivot_accumulator.join(pivot_df, on='SheetName')

    # Update the dictionary with the pivoted DataFrame
    sheets_dict[sheet_name] = pivot_accumulator
